In [0]:
!pip install arrow
!pip install sklearn
import pandas as pd
import arrow
import statsmodels.api as sm
from sklearn import linear_model

In [0]:
import pandas as pd

DATA_URL = 'http://128.138.93.164/aprd6342/data/TravelPonyFacebook.csv'
df = pd.read_csv(DATA_URL)
df.head(10)

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.000000,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.000000,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.000000,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.000000,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.590000,2.512857
5,2013-11-22,2013-11-22,Bob Jackson,TravelPony: Conversion Test,7096,1.018743,7229,32,32,45.01,2,5,3,0,0,0,0,3,22.505000,9.002000
6,2013-11-21,2013-11-21,Bob Jackson,TravelPony: Conversion Test,29437,1.000000,29437,100,80,200.00,10,43,33,0,0,0,0,33,20.000000,4.651163
7,2013-11-21,2013-11-21,Bob Jackson,TravelPony: General Ads,30320,1.018239,30873,178,157,98.09,5,50,45,0,0,0,0,45,19.618000,1.961800
8,2013-11-20,2013-11-20,Bob Jackson,TravelPony: Conversion Test,25951,1.026820,26647,133,113,200.00,8,50,42,0,0,0,0,42,25.000000,4.000000
9,2013-11-20,2013-11-20,Bob Jackson,TravelPony: General Ads,49038,1.000000,49038,324,276,178.77,13,112,99,0,0,0,0,99,13.751538,1.596161


# 1. Create the analytics 'cost per impression' by dividing Amount Spent / Impressions

In [0]:
# create a new column CPI and calculate the same using amount spent/impressions
df['Cost Per Impression'] = df['Amount Spent (USD)'] / df['Impressions']
#empty list dayofweek
dayofweek = []
#Extract the day of the weeek as a numerical value from the start date field and store it in the list dayOfWeek
for i, row in df.iterrows():
    
    x = (arrow.get(str(df['Start Date'][i]), 'YYYY-MM-DD')).format('dddd')
    
    dayofweek.append(x)
    ##dt = arrow.get(row['Start Date'], 'YYYY-MM-DD').reset_index()
    ##x = dt.isoweekday()
    ##dayOfWeek.append(x)


#Calculate the best and worst days of the week based on the mean of CPI   
df['Day of Week'] = dayofweek
r = pd.DataFrame(df.groupby(['Day of Week']).mean()['Cost Per Impression'].sort_values(ascending=False)).reset_index()
print(r)
print(r['Day of Week'][6],"works best.")
print(r['Day of Week'][0],"works worst.")

  Day of Week  Cost Per Impression
0      Friday             0.004097
1      Sunday             0.003688
2    Thursday             0.003398
3     Tuesday             0.003040
4   Wednesday             0.002981
5      Monday             0.002910
6    Saturday             0.002629
Saturday works best.
Friday works worst.


# 2.Compute the correlations

In [0]:
# compute the correlations
data_corr=df.corr()
reach_amtSpent=data_corr['Reach']['Amount Spent (USD)']
freq_amtSpent=data_corr['Frequency']['Amount Spent (USD)']
unqClicks_amtSpent=data_corr['Unique Clicks']['Amount Spent (USD)']
pgLikes_amtSpent=data_corr['Page Likes']['Amount Spent (USD)']
data={'variable':['Reach','Frequency','Unique Clicks','Page Likes'],'correlation':[reach_amtSpent,freq_amtSpent,unqClicks_amtSpent,pgLikes_amtSpent]}
corr_df=pd.DataFrame(data,columns=['variable','correlation'])

print(df[['Amount Spent (USD)','Reach']].corr())
print(df[['Amount Spent (USD)','Frequency']].corr())
print(df[['Amount Spent (USD)','Unique Clicks']].corr())
print(df[['Amount Spent (USD)','Page Likes']].corr())
print("There is the srtongest correlation between Amount Spent (USD) and", corr_df['variable'][corr_df['correlation'].idxmax()],"=",corr_df['correlation'][corr_df['correlation'].idxmax()])

                    Amount Spent (USD)     Reach
Amount Spent (USD)            1.000000  0.703124
Reach                         0.703124  1.000000
                    Amount Spent (USD)  Frequency
Amount Spent (USD)            1.000000   0.130201
Frequency                     0.130201   1.000000
                    Amount Spent (USD)  Unique Clicks
Amount Spent (USD)            1.000000       0.882993
Unique Clicks                 0.882993       1.000000
                    Amount Spent (USD)  Page Likes
Amount Spent (USD)            1.000000    0.757612
Page Likes                    0.757612    1.000000
There is the srtongest correlation between Amount Spent (USD) and Unique Clicks = 0.8829931774784341


In [0]:
# summary the correlations
print(df[['Amount Spent (USD)','Reach','Frequency','Unique Clicks','Page Likes']].corr())
print("The correlation between Amount Spent and Unique Clicks is the strongest with the value of 0.882993. Practically speaking, more unique clicks will most significantly increase the amount spent. ")

                    Amount Spent (USD)     Reach  ...  Unique Clicks  Page Likes
Amount Spent (USD)            1.000000  0.703124  ...       0.882993    0.757612
Reach                         0.703124  1.000000  ...       0.722249    0.304388
Frequency                     0.130201  0.334101  ...       0.135103    0.000182
Unique Clicks                 0.882993  0.722249  ...       1.000000    0.584614
Page Likes                    0.757612  0.304388  ...       0.584614    1.000000

[5 rows x 5 columns]
The correlation between Amount Spent and Unique Clicks is the strongest with the value of 0.882993. Practically speaking, more unique clicks will most significantly increase the amount spent. 


# 3. Simple Multiple Regression Analysis

In [0]:
# 3. Perform a simple multiple regression analysis
UC = df["Unique Clicks"]
RF = df[["Reach","Frequency"]]

model = sm.OLS(UC, RF).fit()
# make the predictions by the model
predictions = model.predict(RF) 

model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Thu, 14 Nov 2019   Prob (F-statistic):                        0.00
Time:                        01:05:43   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
# Maybe bot useful, but good modle to evaluate my conclusion
x1=df[['Reach','Frequency']]
y1=df['Unique Clicks']
regr = linear_model.LinearRegression()
regr.fit(x1, y1)

print ("Using Linear_model from sklearn")
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
    
#predicted Y for the regression model
predictions = regr.predict(x1)
print(predictions)

# r2 value
print(" R squared value is ", regr.score(x1,y1))

Using Linear_model from sklearn
Intercept: 
 29.227730390224757
Coefficients: 
 [ 2.06474936e-03 -2.50632823e+01]
[545.46559791 433.95502979 519.77018255 ...   4.39363526   4.45970724
  11.70488637]
 R squared value is  0.534339731180361


In [0]:
print("Frequency most strongly predicts unique clicks since it has a greater co efficient.It means as the Frequency increases by 1, the predicted value of Unique Clicks increases by 3.6139.")

Frequency most strongly predicts unique clicks since it has a greater co efficient.It means as the Frequency increases by 1, the predicted value of Unique Clicks increases by 3.6139.
